# Entrenamiento y Evaluación de Modelos

## Trabajo Práctico Nro. 2 - Grupo 3

#### Integrantes:
* Ignacio Busso
* Lucas Copes
* Jesica Heit

#### Dataset: https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction
* Detalle: contiene datos de la satisfacción de los pasajeros de diferentes vuelos tomando en cuenta multiples aspectos (calidad del servicio, comodidad, limpieza, etc.)
* Target: columna 'satisfaction', para determinar la satisfacción de un pasajero respecto a un vuelo.
* Dimensiones: 25 columnas x 129.880 filas.

In [ ]:
%matplotlib inline

import warnings
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 0

#Cambios en el estilo de los graficos
plt.style.use('fast')

np.set_printoptions(suppress=True)

warnings.filterwarnings('ignore')

In [ ]:
# Lectura y concatenación de los .csv
train = pd.read_csv('data/train.csv', index_col=[0])
test = pd.read_csv('data/test.csv', index_col=[0])
all_data = pd.concat([train, test], sort=False)

# Asignamos nuevos nombres a algunas de las columnas
new_column_names = {
    'Gender': 'gender',
    'Customer Type': 'customer_type',
    'Age': 'age',
    'Type of Travel': 'business_travel',
    'Class': 'ticket_class',
    'Flight Distance': 'flight_distance',
    'Inflight wifi service': 'wifi_service',
    'Departure/Arrival time convenient': 'departure_arrival_time_convenient',
    'Ease of Online booking': 'online_booking',
    'Gate location': 'gate_location',
    'Food and drink': 'food_and_drink',
    'Online boarding': 'online_boarding',
    'Seat comfort': 'seat_comfort',
    'Inflight entertainment': 'inflight_entertainment',
    'On-board service': 'onboard_service',
    'Leg room service': 'leg_room',
    'Baggage handling': 'baggage_handling',
    'Checkin service': 'checkin',
    'Inflight service': 'inflight_service',
    'Cleanliness': 'cleanliness',
    'Departure Delay in Minutes': 'departure_delay',
    'Arrival Delay in Minutes': 'arrival_delay',
}

all_data.rename(columns=new_column_names, inplace=True)
all_data.set_index('id', inplace=True)

## Métrica

Accuracy es una métrica para evaluar los modelos de clasificación. Mide el porcentaje de casos donde el modelo acertó. Es una de las métricas más usadas.

Nuestro grupo decide utilizar esta métrica ya que la información no se encuentra desbalanceada debido a que disponemos de un 56% de personas que se encuentran “neutral or dissatisfied” y un 43% de personas que se encuentran “satisfied”. Debido a que nuestras clases se encuentran muy bien balanceadas, Accuracy es una buena métrica a utilizar.

## Preprocesado
1. Conversión de variables cualitativas a booleanas cuantitativas.
2.  
    - Limpieza de registros con nulos implícitos en los servicios. Aplica solo a features con menos de 500 casos.
    - Limpieza de registros NaN en `arrival_delays`. (~400 registros)
3. Limpieza de valores extremos en `arrival_delays` y `departure_delays`. (~600 registros)
4. Limpieza de valores extremos en `flight_distance`. Aplica solo a vuelos de más de 4000 unidades de longitud (~75 registros)

In [ ]:
# Conversión a variables booleanas
all_data['gender'] = all_data['gender'].replace(['Male','Female'],[0,1])
all_data['customer_type'] = all_data['customer_type'].replace(['disloyal Customer','Loyal Customer'],[0,1])
all_data['business_travel'] = all_data['business_travel'].replace(['Personal Travel','Business travel'],[0,1])
all_data['satisfaction'] = all_data['satisfaction'].replace(['neutral or dissatisfied','satisfied'],[0,1])

# Limpieza de filas con pocas (< 500) features de servicios nulas (== 0) y arrivals NaNs
all_data = all_data[
    ~(all_data == 0).gate_location &
    ~(all_data == 0).food_and_drink &
    ~(all_data == 0).seat_comfort & 
    ~(all_data == 0).inflight_entertainment &
    ~(all_data == 0).onboard_service &
    ~(all_data == 0).checkin &
    ~(all_data == 0).inflight_service &
    ~(all_data == 0).cleanliness &
    ~all_data.arrival_delay.isnull()
    ]

# Limpieza de valores extremos en Arrivals y Departures
all_data = all_data.drop(all_data[all_data.arrival_delay > 240].index)
all_data = all_data.drop(all_data[all_data.departure_delay > 240].index)

# Limpieza de valores extremos en flight_distance (> 4000 - 75 registros)
all_data = all_data.drop(all_data[all_data.flight_distance > 4000].index)

### Feature Engineering
Se crean una serie de intervalos numéricos para cuatro variables. Los valores son reemplazados por el mayor número de cada intervalo. Ej: Si un viaje es de 150km, el valor es reemplazado por 500km, ya que cae dentro del intervalo de 0 a 500.
Variables involucradas: `age`, `flight_distance`, `departure_delay` y `arrival_delay`

In [ ]:
# Creación de intervalos para features numéricas
    # Puede ser visto como Binning / Redondeo de números? Feature Engineering?
    # Ver de dejarlo para el final. Ejecutarlo por separado en otro df y comparar resultados.

all_data_fe = all_data.copy(deep=True)

# ¿Corresponde que los labels tengan como nombre el mayor numero del intervalo? ¿Debería usar un valor calculado en su lugar? (Media/Mediana/Moda)
all_data_fe["age_interval"] = pd.cut(x=all_data_fe['age'], bins=[0,12,18,30,60,80,100], labels=[12,18,30,60,80,100])
all_data_fe["flight_distance_interval"] = pd.cut(x=all_data_fe['flight_distance'], bins=[0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000], labels=[500,1000,1500,2000,2500,3000,3500,4000,4500,5000])
all_data_fe["departure_delay_interval"] = pd.cut(x=all_data_fe['departure_delay'], bins=[-1,0,15,30,45,60,120,180,240], labels=[0,15,30,45,60,120,180,240])
all_data_fe["arrival_delay_interval"] = pd.cut(x=all_data_fe['arrival_delay'], bins=[-1,0,15,30,45,60,120,180,240], labels=[0,15,30,45,60,120,180,240])

### Armado de datasets
- 60% Train         (77k)
- 20% Validation    (25k)
- 20% Test          (25k)

In [ ]:
from sklearn.model_selection import train_test_split

def armado_datasets(df):
    train, not_train = train_test_split(df, test_size=0.4, random_state=7)
    validation, test = train_test_split(not_train, test_size=0.5, random_state=7)

    train.shape, validation.shape, test.shape
    return train, validation, test

train, validation, test = armado_datasets(all_data)
train_fe, validation_fe, test_fe = armado_datasets(all_data_fe)

### DataFrameMapper
- Escalar todas las variables numéricas no booleanas con MinMaxScaler (valores entre 0-1).
- Imputar las features con más de 600 nulos con IterativeImputer.

In [ ]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, QuantileTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Imputar y escalar
mapper = DataFrameMapper([
    (['gender'], None),
    (['customer_type'], None),
    (['age'], [MinMaxScaler()]),
    (['business_travel'], None),
    (['ticket_class'], [OneHotEncoder()]),
    (['flight_distance'], [MinMaxScaler()]),
], df_out=True)
mapper.fit(train)

mapper_fe = DataFrameMapper([
    (['gender'], None),
    (['customer_type'], None),
    (['age_interval'], [MinMaxScaler()]),
    (['business_travel'], None),
    (['ticket_class'], [OneHotEncoder()]),
    (['flight_distance_interval'], [MinMaxScaler()]),
], df_out=True)
mapper_fe.fit(train_fe)

mapper_fe_qt = DataFrameMapper([
    (['gender'], None),
    (['customer_type'], None),
    (['age'], [QuantileTransformer(), MinMaxScaler()]),
    (['business_travel'], None),
    (['ticket_class'], [OneHotEncoder()]),
    (['flight_distance'], [QuantileTransformer(), MinMaxScaler()]),
], df_out=True)
mapper_fe_qt.fit(train)

#mfit = mapper.fit(all_data)
#all_data_transformed = mapper.transform(all_data)

In [ ]:
# Transformación de un sample:
sample = train.sample(3, random_state=7)
sample_fe = train_fe.sample(3, random_state=7)
sample_fe_qt = train.sample(3, random_state=7)
# Sample original:
sample

In [ ]:
mapper.transform(sample)

In [ ]:
mapper_fe.transform(sample_fe)

In [ ]:
mapper_fe_qt.transform(sample_fe_qt)

## Modelos
Serán utilizados los siguientes algoritmos:
1. Regresión Logística
2. Arboles de Decisión
3. Gradient Boosting

En este punto se entrenan y evalúan los modelos elegidos utilizando tanto parámetros estándar como algunos de los hiperparámetros principales de cada uno de ellos. Al finalizar, en base a las métricas y el comportamiento general se arriba a conclusiones.

In [ ]:
from collections import defaultdict
from sklearn import metrics

def evaluate_model(model, feature_engineering=False, test=False ,set_names=('train', 'validation'), title='', show_cm=True):
    if title:
        display(title)
        
    final_metrics = defaultdict(list)

    if feature_engineering:
        set_names=('train_fe', 'validation_fe')
    else:
        set_names=('train', 'validation')
        
    if test:
        set_names=('train', 'validation', 'test')
    
    if show_cm:
        fig, axis = plt.subplots(1, len(set_names), sharey=True, figsize=(15, 3))
    
    for i, set_name in enumerate(set_names):
        if not feature_engineering:
            assert set_name in ['train', 'validation', 'test']
        else:
            assert set_name in ['train_fe', 'validation_fe', 'test_fe']
        
        set_data = globals()[set_name]

        y = set_data.satisfaction
        y_pred = model.predict(set_data)
        final_metrics['Accuracy'].append(metrics.accuracy_score(y, y_pred))
        final_metrics['Precision'].append(metrics.precision_score(y, y_pred))
        final_metrics['Recall'].append(metrics.recall_score(y, y_pred))
        final_metrics['F1'].append(metrics.f1_score(y, y_pred))
        
        if show_cm:
            ax = axis[i]
            sns.heatmap(metrics.confusion_matrix(y, y_pred), ax=ax, cmap='Blues', annot=True, fmt='.0f', cbar=False)

            ax.set_title(set_name)
            ax.xaxis.set_ticklabels(['Not Satisfied', 'Satisfied'])
            ax.yaxis.set_ticklabels(['Not Satisfied', 'Satisfied'])
            ax.set_xlabel('Predicted class')
            ax.set_ylabel('True class')

    display(pd.DataFrame(final_metrics, index=set_names))
    if show_cm:
        plt.tight_layout()
        plt.show()

### Regresión Logística

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

#### Regresión Logística - Parámetros estándar

In [ ]:
# standard
regression_model = LogisticRegression(random_state=7)
lr_model = Pipeline([
    ('mapper', mapper),
    ('classifier', regression_model),
])

lr_model.fit(train, train.satisfaction)
evaluate_model(lr_model, title='Regresión Lineal')

#### Regresión Logística - Iteraciones máximas

Entrenamiento limitando la cantidad de iteraciones máximas que el algoritmo puede hacer antes de converger.

In [ ]:
# max_iter = 2
regression_model = LogisticRegression(max_iter=2, random_state=7)
lr_model = Pipeline([
    ('mapper', mapper),
    ('classifier', regression_model),
])

lr_model.fit(train, train.satisfaction)
evaluate_model(lr_model, title='Regresión Lineal - max_iter = 2')

La métrica Accuracy cae ~3 puntos porcentuales en este caso, ya que al limitar la cantidad de iteraciones forzamos al algoritmo a converger antes de tiempo, cuando todavía hay lugar para mejores resultados.

### Árboles de Decisión

In [ ]:
from sklearn.tree import DecisionTreeClassifier

Función para graficar los arboles:

In [ ]:
import graphviz  # pip install graphviz
from sklearn.tree import export_graphviz

def graph_tree(tree, col_names):
    graph_data = export_graphviz(
        tree, 
        out_file=None, 
        feature_names=col_names,  
        class_names=['Neutral/Not Satisfied', 'Satisfied'],  
        filled=True, 
        rounded=True,  
        special_characters=True,
    )
    graph = graphviz.Source(graph_data)  
    return graph

#### Árboles de Decisión - Parámetros estándar

In [ ]:
# standard
tree_model = DecisionTreeClassifier(random_state=7)
dt_model = Pipeline([
    ('mapper', mapper),
    ('classifier', tree_model),
])

dt_model.fit(train, train.satisfaction)
evaluate_model(dt_model, title='Decision Tree')

print('Cantidad de nodos:', tree_model.tree_.node_count)
print('Profundidad máxima:', tree_model.tree_.max_depth)

El resultado del entrenamiento sin definir parámetros es un overfitting bastante claro en Train, con todas las métricas por encima del 97%. Esto se debe principalmente al hecho de que no se le ha definido una profundidad máxima, permitiendo así que el arbol siga memorizando los ejemplos y creando una rama para cada uno de ellos.

Puede verse que el arbol llega a tener una profundidad maxima de 56 niveles y un total de 46033 (59% de los registros del dataset de Train).

#### Árboles de Decisión - Profundidad máxima
**Entrenamiento definiendo un `max_depth` de 4 (Default = None).** De esta manera buscamos eliminar el overfitting visto en el caso anterior.

In [ ]:
# max_depth = 4
tree_model = DecisionTreeClassifier(max_depth=4,random_state=7)
dt_model = Pipeline([
    ('mapper', mapper),
    ('classifier', tree_model),
])

dt_model.fit(train, train.satisfaction)
evaluate_model(dt_model, title='Decision Tree - max_depth = 4')

print('Cantidad de nodos:', tree_model.tree_.node_count)
print('Profundidad máxima:', tree_model.tree_.max_depth)

Ya con un límite en la cantidad máxima de niveles permitida se alcanzan valores más realistas.

**Entrenamiento definiendo un `max_depth` de 1 (valor mínimo).**

In [ ]:
# max_depth = 1
tree_model = DecisionTreeClassifier(max_depth=1,random_state=7)
dt_model = Pipeline([
    ('mapper', mapper),
    ('classifier', tree_model),
])

dt_model.fit(train, train.satisfaction)
evaluate_model(dt_model, title='Decision Tree - max_depth = 1')

print('Cantidad de nodos:', tree_model.tree_.node_count)
print('Profundidad máxima:', tree_model.tree_.max_depth)

Utilizar el valor mínimo de profundidad acarrea a que el Accuracy caiga 4 puntos porcentuales. Naturalmente, esto ocurre ya que acotamos demasiado la cantidad de cortes que puede hacer el árbol de decisión.

En el siguiente gráfico podemos ver que la clasificación se hace solamente en base al valor de la feature `online_boarding`:

In [ ]:
#graph_tree(tree_model, mapper.transformed_names_)

### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

#### Gradient Boosting - Entrenamiento con parámetros estándar

In [ ]:
# standard
gradient_model = GradientBoostingClassifier(random_state=7)
gb_model = Pipeline([
    ('mapper', mapper),
    ('classifier', gradient_model),
])

gb_model.fit(train, train.satisfaction)
evaluate_model(gb_model, title='Gradient Boosting')

#### Gradient Boosting - Learning rate
Entrenamiento con un `learning_rate` mucho mayor (20, en lugar del 0.1 por default).

In [ ]:
# learning_rate = 20
gradient_model = GradientBoostingClassifier(learning_rate=20, random_state=7)
gb_model = Pipeline([
    ('mapper', mapper),
    ('classifier', gradient_model),
])

gb_model.fit(train, train.satisfaction)
evaluate_model(gb_model, title='Gradient Boosting - learning_rate = 20')

Esto resulta en una métrica Accuracy 50 puntos porcentuales más baja que la anterior, ya que el modelo falla y clasifica erroneamente a muchos de los casos. Por otro lado, el tiempo de ejecución se ve afectado solo mínimamente.

#### Gradient Boosting - Cantidad de árboles
**Entrenamiento definiendo un `n_estimators` de 4 (Default = 100).** Este parámetro indica la cantidad de arboles que Gradient Boosting va a modelar.

In [ ]:
# n_estimators = 4
gradient_model = GradientBoostingClassifier(n_estimators=4, random_state=7)
gb_model = Pipeline([
    ('mapper', mapper),
    ('classifier', gradient_model),
])

gb_model.fit(train, train.satisfaction)
evaluate_model(gb_model, title='Gradient Boosting - n_estimators = 4')

En este caso las métricas dan apenas uno o dos puntos porcentuales menos que la ejecución inicial sin parámetros. Esto parece indicar que el rendimiento del modelo llega a una meseta a partir de los 4 árboles creados.

**Entrenamiento definiendo un `n_estimators` de 3 (Default = 100).**

In [ ]:
# n_estimators = 2
gradient_model = GradientBoostingClassifier(n_estimators=2, random_state=7)
gb_model = Pipeline([
    ('mapper', mapper),
    ('classifier', gradient_model),
])

gb_model.fit(train, train.satisfaction)
evaluate_model(gb_model, title='Gradient Boosting - n_estimators = 2')

En esta segunda prueba y con solo dos árboles creados, las métricas se mantienen muy similares al ejemplo anterior. Esto refuerza la teoría de que se forma una meseta a partir de, ahora, el segundo árbol utilizado.

## Overfitting

En esta sección se evalúa cada uno de los tres algoritmos elegidos modificando continuamente alguno de sus hiperparámetros principales a lo largo de varias iteraciones, mientras se compara el valor de la métrica Accuracy en los datasets Train y Validation. En base a esta información se selecciona la configuración óptima de hiperparámetros para los modelos a entrenar en el resto del trabajo.

In [ ]:
def overfitting_test(model, set_names=('train', 'validation')):
    final_metrics = defaultdict(list)
    for i, set_name in enumerate(set_names):
        assert set_name in ['train', 'validation', 'test']
        set_data = globals()[set_name]

        y = set_data.satisfaction
        y_pred = model.predict(set_data)
        final_metrics['Accuracy'].append(metrics.accuracy_score(y, y_pred))

    return final_metrics

### Regresión Logística
- Parámetro modificado: `max_iter`
- Valores: 1 a 20

In [ ]:
train_acc = []
validation_acc = []
for i in range(1,21):
    regression_model = LogisticRegression(max_iter=i, random_state=7)
    lr_model = Pipeline([
        ('mapper', mapper),
        ('classifier', regression_model),
    ])
    lr_model.fit(train, train.satisfaction)

    final_metrics = overfitting_test(lr_model)
    train_acc.append(final_metrics['Accuracy'][0])
    validation_acc.append(final_metrics['Accuracy'][1])

plt.figure(figsize=(12,8))
plt.plot(train_acc, label='Train')
plt.plot(validation_acc, label='Validation')
plt.axvline(x=6, color='red', linestyle=':') # 6 es x=7
plt.title('Accuracy respecto a la cantidad máxima de iteraciones')
plt.ylabel('Accuracy')
plt.xlabel('Cantidad máxima de iteraciones')
plt.xticks(np.arange(len(train_acc)), np.arange(1, len(train_acc)+1))
plt.legend(loc='lower right')
plt.grid()
plt.show()

Para el modelo final se realiza el corte del entrenamiento al llegar a las 7 iteraciones, punto máximo antes de que la performance caiga y se vuelva un tanto inestable.

### Árbol de Decisión
- Parámetro modificado: `max_depth`
- Valores: 1 a 20

In [ ]:
train_acc = []
validation_acc = []
for i in range(1,21):
    tree_model = DecisionTreeClassifier(max_depth=i, random_state=7)
    dt_model = Pipeline([
        ('mapper', mapper),
        ('classifier', tree_model),
    ])
    dt_model.fit(train, train.satisfaction)

    final_metrics = overfitting_test(dt_model)
    train_acc.append(final_metrics['Accuracy'][0])
    validation_acc.append(final_metrics['Accuracy'][1])

plt.figure(figsize=(12,8))
plt.plot(train_acc, label='Train')
plt.plot(validation_acc, label='Validation')
plt.axvline(x=5, color='red', linestyle=':') # 5 es x=6
plt.title('Accuracy respecto a la profundidad máxima')
plt.ylabel('Accuracy')
plt.xlabel('Profundidad máxima')
plt.xticks(np.arange(len(train_acc)), np.arange(1, len(train_acc)+1))
plt.legend(loc='lower right')
plt.grid()
plt.show()

Para el modelo final se realiza el corte a la profundidad máxima de 6, antes de que la divergencia entre los Accuracy de Train y Validation continúe acrecentándose.

### Gradient Boosting
- Parámetro modificado: `n_estimators`
- Valores: 1 a 20

In [ ]:
train_acc = []
validation_acc = []
for i in range(1,21):
    gradient_model = GradientBoostingClassifier(n_estimators=i, random_state=7)
    gb_model = Pipeline([
        ('mapper', mapper),
        ('classifier', gradient_model),
    ])
    gb_model.fit(train, train.satisfaction)

    final_metrics = overfitting_test(gb_model)
    train_acc.append(final_metrics['Accuracy'][0])
    validation_acc.append(final_metrics['Accuracy'][1])

plt.figure(figsize=(12,8))
plt.plot(train_acc, label='Train')
plt.plot(validation_acc, label='Validation')
plt.axvline(x=1, color='red', linestyle=':') # 1 es x=2
plt.title('Accuracy respecto a la cantidad máxima de árboles')
plt.ylabel('Accuracy')
plt.xlabel('Cantidad máxima de árboles')
plt.xticks(np.arange(len(train_acc)), np.arange(1, len(train_acc)+1))
plt.legend(loc='lower right')
plt.grid()
plt.show()

Para el modelo final se realiza el corte al llegar a 2 árboles creados, y el max_depth se mantiene por default en 3. No tiene sentido utilizar una cantidad mayor ya que las metricas tienden a una meseta total en iteraciones posteriores.

## Modelos definitivos

### Regresión Logística sin Feature Engineering

In [ ]:
# max_iter = 10
regression_model = LogisticRegression(max_iter=10, random_state=7)
lr_model = Pipeline([
    ('mapper', mapper),
    ('classifier', regression_model),
])

lr_model.fit(train, train.satisfaction)
evaluate_model(lr_model, title='Regresión Lineal sin Feature Engineering - max_iter = 10')

### Regresión Logística con Feature Engineering

In [ ]:
# max_iter = 10
regression_model = LogisticRegression(max_iter=10, random_state=7)
lr_model_fe = Pipeline([
    ('mapper', mapper_fe),
    ('classifier', regression_model),
])

lr_model_fe.fit(train_fe, train_fe.satisfaction)
evaluate_model(lr_model_fe, feature_engineering=True, title='Regresión Lineal con Feature Engineering (intervalos creados a mano) - max_iter = 10')

### Regresión Logística con Feature Engineering - Quantile Transformation

In [ ]:
# max_iter = 10
regression_model = LogisticRegression(max_iter=10, random_state=7)
lr_model_fe_qt = Pipeline([
    ('mapper', mapper_fe_qt),
    ('classifier', regression_model),
])

lr_model_fe_qt.fit(train, train.satisfaction)
evaluate_model(lr_model_fe_qt, title='Regresión Lineal con Feature Engineering (Quantile Transformation) - max_iter = 10')

### Comparación

In [ ]:
evaluate_model(lr_model, title='Regresión Logística sin Feature Engineering', show_cm=False)
evaluate_model(lr_model_fe,feature_engineering= True, title='Regresión Logística con Feature Engineering - Intervalos a mano', show_cm=False)
evaluate_model(lr_model_fe_qt, title='Regresión Logística con Feature Engineering - Quantile Transformation', show_cm=False)

### Árbol de decisión sin Feature Engineering

In [ ]:
# max_depth = 10
tree_model = DecisionTreeClassifier(max_depth=10,random_state=7)
dt_model = Pipeline([
    ('mapper', mapper),
    ('classifier', tree_model),
])

dt_model.fit(train, train.satisfaction)
evaluate_model(dt_model, title='Árbol de Decisión sin Feature Engineering - max_depth = 10')

print('Cantidad de nodos:', tree_model.tree_.node_count)
print('Profundidad máxima:', tree_model.tree_.max_depth)

### Árbol de decisión con Feature Engineering 

In [ ]:
# max_depth = 10
tree_model = DecisionTreeClassifier(max_depth=10,random_state=7)
dt_model_fe = Pipeline([
    ('mapper', mapper_fe),
    ('classifier', tree_model),
])

dt_model_fe.fit(train_fe, train_fe.satisfaction)
evaluate_model(dt_model_fe, feature_engineering=True, title='Árbol de Decisión con Feature Engineering (intervalos creados a mano) - max_depth = 10')

print('Cantidad de nodos:', tree_model.tree_.node_count)
print('Profundidad máxima:', tree_model.tree_.max_depth)

### Árbol de decisión con Feature Engineering - Quantile Transformation

In [ ]:
# max_depth = 10
tree_model = DecisionTreeClassifier(max_depth=10,random_state=7)
dt_model_fe_qt = Pipeline([
    ('mapper', mapper_fe_qt),
    ('classifier', tree_model),
])

dt_model_fe_qt.fit(train, train.satisfaction)
evaluate_model(dt_model_fe_qt, title='Árbol de Decisión con Feature Engineering (Quantile Transformation) - max_depth = 10')

print('Cantidad de nodos:', tree_model.tree_.node_count)
print('Profundidad máxima:', tree_model.tree_.max_depth)

### Comparación

In [ ]:
evaluate_model(dt_model, title='Árbol de Decisión sin Feature Engineering', show_cm=False)
evaluate_model(dt_model_fe,feature_engineering= True, title='Árbol de Decisión con Feature Engineering - Intervalos a mano', show_cm=False)
evaluate_model(dt_model_fe_qt, title='Árbol de Decisión con Feature Engineering - Quantile Transformation', show_cm=False)

### Gradient Boosting sin Feature Engineering

In [ ]:
# n_estimators = 4
gradient_model = GradientBoostingClassifier(n_estimators=4, random_state=7)
gb_model = Pipeline([
    ('mapper', mapper),
    ('classifier', gradient_model),
])

gb_model.fit(train, train.satisfaction)
evaluate_model(gb_model, title='Gradient Boosting sin Feature Engineering - n_estimators = 4')

### Gradient Boosting con Feature Engineering

In [ ]:
# n_estimators = 4
gradient_model = GradientBoostingClassifier(n_estimators=4, random_state=7)
gb_model_fe = Pipeline([
    ('mapper', mapper_fe),
    ('classifier', gradient_model),
])

gb_model_fe.fit(train_fe, train_fe.satisfaction)
evaluate_model(gb_model_fe, feature_engineering=True, title='Gradient Boosting con Feature Engineering (intervalos creados a mano) - n_estimators = 4')

### Gradient Boosting con Feature Engineering - Quantile Transformation

In [ ]:
# n_estimators = 4
gradient_model = GradientBoostingClassifier(n_estimators=4, random_state=7)
gb_model_fe_qt = Pipeline([
    ('mapper', mapper_fe_qt),
    ('classifier', gradient_model),
])

gb_model_fe_qt.fit(train, train.satisfaction)
evaluate_model(gb_model_fe_qt, title='Gradient Boosting con Feature Engineering (Quantile Transformation) - n_estimators = 4')

### Comparación

In [ ]:
evaluate_model(gb_model, title='Gradient Boosting sin Feature Engineering', show_cm=False)
evaluate_model(gb_model_fe,feature_engineering= True, title='Gradient Boosting con Feature Engineering - Intervalos a mano', show_cm=False)
evaluate_model(gb_model_fe_qt, title='Gradient Boosting con Feature Engineering - Quantile Transformation', show_cm=False)

### Conclusión

Al examinar los tres modelos utilizando tres diferentes variantes del mismo Dataset, las métricas muestran que el uso de Quantile Transformation como Feature Engineering no aporta ninguna ventaja por sobre el Dataset estándar, mientras que el Feature Engineering manual por intervalos emperoa las métricas ligeramente. Por lo tanto, se decide realizar la evaluación de los modelos utilizando el Dataset estándar.

### Comparación final entre modelos

In [ ]:
evaluate_model(lr_model, title='Regresión Logística', show_cm=False)
evaluate_model(dt_model, title='Árbol de Decisión', show_cm=False)
evaluate_model(gb_model, title='Gradient Boosting', show_cm=False)

In [ ]:
chosen_model = dt_model
evaluate_model(chosen_model, test= True, title='Modelo elegido', set_names=('train', 'validation', 'test'), show_cm=False)

## Conclusión

Luego de analizar los distintos métodos utilizados, podemos ver que según qué variables de entrada utilizamos para entrenar nuestro modelo, los resultados pueden variar bastante. Si elegimos como entrada las variables del puntaje de cada servicio ofrecido, las métricas en general funcionan con un Accuracy mayor al 89%. Esto es porque resulta fácil deducir una salida de satisfacción final teniendo en cuenta el puntaje de cada apartado por separado como entrada. 
Ahora, si usamos solo las variables del pasajero y datos básicos del vuelo, podemos ver una situación más cercana al mundo real, ya que no estamos tratando de evaluar el puntaje sino que las variables que se tienen en cuenta no están en un principio tan relacionadas con el target. 